# Replication Baskaran & Hessami (2018)

This notebook contains a replication of [Baskaran, T., & Hessami, Z. (2018) Does the Election of a Female Leader Clear the Way for More Women in Politics? *American Economic Journal: Economic Policy*, 10(3): 91-121](https://www.aeaweb.org/articles?id=10.1257/pol.20170045).

For this purpose, first, I present a executive summary on the research hypothesis, analysed data and the method used for this purpose. However, I strongly recommend that one reads the original article prior to reading this notebook. Second, descriptive and graphical data analyses are presented. This is useful for understanding and making sense of the data that is used for the empirical analysis and serves as a preparation for the third and main part of this notebook: A replication of the key results of the paper. Besides the baseline results of the main model this includes econometric analyses regarding the internal validity of the used method as well as the robustness tests presented in the article. Finally, the fourth section of this notebook concludes the results of this exercise.

Naturally, I draw heavily from the original articel throughout this replication. Unless stated otherwise, the reader should therefore assume that the ideas presented within this notebook reproductions of the contents of Baskaran & Hessami (2018). 

Overall, the results presented by Baskeran & Hessami (2018) can successfully be replicated. Particulary, the baseline results of the main estimation model, the testresults for internal validity of the econometric method, and the results for robustness tests do not give reason to take the credibility of this article into question. 


## 1. Excecutive summary of the paper
* **Theoretical Framework:** *Anti-female voter bias* results in underrepresentation of females in politics.


* **Research Hypothesis:** Exposure to a female mayor deminishes *anti-female biases* of voters.

**Figure 1: Causal Graph**
![Error: here should be figure 1](img/causal_graph.png)

* **Identification problems:**

    * Other potential drivers of the underrepresentation of femeales in politics:
        - Political parties influence election outcomes. Thus, underrepresentation might be caused by *anti-female party biases*
        - Parties might anticipate *anti-female voter bias* and, therefore, are less willing to field female candidates in competitive races.
        - Women who expect discrimination might be reluctand to run for election/ pursue a career in politics in the first place. 

    * Isolating *anti-female voter bias*: Rule out alternative explanations
        - (a) Initial rank of female candidates does not impact their rank advancements
        - (b) Characteristics of women on party's list chage only little in case there is a female mayor in office
        - (c) Number of female candidates on list remains unchanged
        - (a), (b), and (c) indicate little *anti-female party bias*
        

* **Data:**

    * Data on local council election in the German state of Hesse.
        - Open-list electoral system (parties specify order of candidates, voters can distribute preferential votes to individual candidates).
        - Includes gender, list ranks, personal characteristics among other information for ~109.017 candidates who competed in 2001, 2006, 2011, or 2016 elections
    
    * Data on mayor elections for all 426 municipalities in Hesse
        - including the elected mayor's gender and the margin the election was won by
        - including characteristcs of each municipalities 
    
    
* **Method:**

    * Authors use a regression discontinuity design (RDD) to study the causal effects of female mayors on subsequent council election results.
    
    * The discontinuity is provided by close mixed-gender mayoral elections.
    
    * estimated model equation: 
     > $rank \, improvement_{k, i, t} = \alpha + \beta female \, mayor_{i, t} + f(vote \, margin)_{i, t} + female \, mayor_{i, t} \times  g(vote \, margin)_{i, t} + \epsilon_{k, i, t}$
     
     Where 
         - $rank \, improvement_{k, i, t}$ is the normalized rank improvement of candidate $k$ in municipality $i$ and local election year $t$.
         - $vote \, margin_{i, t}$ is the margin of victory of the female candidate in the preceeding mayoral election (negative values, thus, denote a loss of the female candidate).
         - $female \, mayor_{i, t}$ is a dummy that takes value 1 if a female candiate has won the mayoral election preceeding the council election $(female \, mayor_{i, t} = 1 \Leftrightarrow vote \, margin_{i, t} > 0)$.





* **Results:**
    * A female candidate winning a close mixed-gender mayoral election causes...
        
        - voters to give preferential votes to female council candidates.
    
        - a 4 percentage points higher share of female council members.
        
        - positive spillover effects on female council candidates in neighboring municipalities.
       

In [ ]:
# Import packages and auxiliary functions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from scipy.stats import ttest_ind

from auxiliary.localreg import *
from auxiliary.auxiliary_functions import * 
from auxiliary.plots_and_figures import *

# General notebooks settings
pd.options.display.max_columns = None

# Settings for matlibplot package
plt.rc('axes', axisbelow=True)

# Import auxiliary datasets
df_mayor_election_data = pd.read_stata('data/mayor_election_data.dta')
df_municipality_characteristics = pd.read_stata('data/municipality_characteristics_data.dta')
df_mayor_election_data = pd.read_stata('data/mayor_election_data.dta')
df_placebo = pd.read_stata('data/dataset_with_lagged_rank_improvments.dta')

# Import main dataset and print a few randomly seleted rows for illustration
df_main_dataset = pd.read_stata('data/main_dataset.dta')
df_main_dataset.sample(5)



## 2. Descriptive and Graphical Analyses
This section presents a descriptive summary of the data set which is provided by the authors. First, the coverage of the data set is described, i.e, how many elections are observed for what number of municipalities and other rather superficial properties. We will have a look at the compositions of this data set, particulary in relation to the subsample of female council candidates which will be of increased interest throughout article and, hence, for this replication.

Second, we will try to make sense of the observable discontinuities in observations. When inspecting the data set, in particular the observations for margin of victory and rank improvement combinations, noticeable discontinuities in both dimensions, abcissa and ordinate can be seen. This is especially interesting, because these discontinuities lead to stacked observations in a scatter plot which, in term, make it hard to judge if there is a discontinuity around the cutoff value of margin of victory.

This section concludes with a summary of the insights that can be gained from this very top level descriptive approach as well as the finding, that a discontinuity around the cutoff yet needs to be identified.

### 2.1. Description of the Dataset
The data set contains observations on 1721 mayor elections. This includes the margin of victory as well as the gender of the mayoral candidates. Out of these 1721 elections, 268 are classified as a *mixed gender election*. The term *mixed gender election* is defined by the authors as "mayor elections \[where\] the top two candidates were of opposite gender".

At the same time, observations on the election results of 109.017 council candidates are provided. When only those council candidates are kept in the sample which ran at an election that was preceeded by a mixed gender mayor election, a total of 23.169 council candidates is observed. For each of these candidates individual characteristics like gender, education and occupation are observed as well as their success in the council election is observed. Success in a council election is measured by the normalized rank improvement, which will be looked at in more detail in subsection 1.2.


In [ ]:
# Count of mayor elections
descriptive_dict = {}
descriptive_dict["mayor_election_count"] = df_mayor_election_data["mayor_election_year"].count()

# Count of mixed gender elections
descriptive_dict["mixed_gender_elections_count"] = df_mayor_election_data["rdd_sample"].sum()

# Count of council candidates
descriptive_dict["council_candidates_count"] = df_main_dataset.gkz_jahr.count().astype(str)

# Count of council candidates in mixed gender election municipality
descriptive_dict["relevant_council_candidates_count"] = df_main_dataset.rdd_sample.count()

# print results
for k in descriptive_dict.keys():
    print(k, ":", descriptive_dict[k], "\n")

In [ ]:
# Generate a Barchart presenting the number of municipalities for which observations are available and for the
# number of candidates per gender.
plot_observations_municipalities_candidates(df_main_dataset)

# Calculate the total number of Municipalities 
muni_count = df_mayor_election_data.gkz.nunique()
print("In total there is data on characteristics available for", 
      "{} different Hessian Municipalities".format(muni_count)
     )

Availability of administrative data on municipalities (i.e. their population, per capita tax revenues, employment among others) is worse in early years. It is available for all 426 municipalities in 2016, however, for in 2001 it is available for 199 municipalities only (below figure, left side). While data for 2016 was provided by The Statistical Office of Hesse, data for previous years come "from varying internet sources such as municipalities' websites as well as e-mail communication with the mayors, their administrative assistants, and high-ranking officials in the local administration." The quality of this information, thus, cannot be assessed any further.

A similar pattern in availablility can be seen for the number of candidates observed in each year. The share of female candidates seems to be relatively stable at approximatly 25 % in each year (below figure, right side). 

TODO: Is there any chances (in particular in the right plot) when looking at candidates that are in the rdd sample, only?

![Error: Here should be figure 1!](out/figure_1.png "Figure 1: Number of observations on municipalities (left) and candidates per year (right).")

When analyzing effects of a female mayor on the rank improvement of a female council candidate, it seems quite intuitive that candidate's characteristics should be ruled out as the reason for differences between candidates in election success. While this will adressed in a more formal way at a later point of the paper (TODO: ADD CHAPTER HERE), it might be insightful to compare characteristics (i.e. age, education and occupation) of the subsample of female candidates only to the characteristics of all candidates. 

In [ ]:
# Calculate a descriptive summary table using a function stored in the auxiliary/plots_and_fígures file
summary_stats(df_main_dataset)

This table compares characteristics of council candidates and their success in the council election between the whole sample and the female subsample. There are noticeable differences in the first row "gewinn_norm" which provides data on the normalized rank improvement of a candidate. By showing a lower (even negative) rank improvement on average, females appear to be less successful in council elections than male candidates. 

While most of the candidates' characteristics seem to be quite similar in both groups, there is some noticeable differences when it comes to the candidates' occupations: While the share of engineers is six times higher in the whole sample then among females, more than four times as many women than men reported domestic work as their occupation (category "housewifehusband").Furthermore, there is smaller but still noticeable differences, e.g. in occupations "selfemployed" and "retired" (both taking higher shares for male candidates) as well as "teacher" and "civil_administration" (both higher for female candidates).

Overall, especially with respect to age and education of the candidates, there seems to be relatively little differences as seen from this descriptive level of inspection.

### 2.2. Discontinuity in Distribution of Observations
In order to get a better understanding of the data provided, the relationship of the margin of victory of the female mayor (abscissa) and the rank improvements of female candidates in the subsequent council election (ordinate) is illustrated (see firgure 2 below). For this exercise, only data on council elections that subceeded mixed gender mayor elections will be used. In addition, only rank improvements of female council candidates are of interest for this analysis and are, therefore, included in the illustration below. 

Note that for easier interpretability *rank improvements* in the council elections are provided in a normalized form, facilitating comparability of changes in ranks of candidates between councils of different sizes. This conversion follows equation (\ref{eq1}) of the article: 

\begin{align}
    rank \, improvement = (\frac{initial \, rank - final \, rank}{council \, size}) \times 100 \label{eq1} \tag{1}
\end{align}



In [ ]:
# Create a dataset that only contains observations on (a) on rank improvements of female council candidates and 
# (b) mixed gender mayor elections.
rdd_data = df_main_dataset.loc[(df_main_dataset.female == 1) & (df_main_dataset.rdd_sample == 1)]

# Create a scatter plot that shows all combinations of the margin of victory of a female in a mixed
# gender mayor lection and the rank improvements of female candidates in subsequent council elections.
# Function is stored in auxiliary/plots_and_figures
plot_observations(df=rdd_data, s=0.1)


## Auxiliary addition: Find the reason for the discrete changes in gewinn_norm. 
# This would make sense, if the council sizes are small and many councils have same sizes. 
# Thus, create a plot showing the distribution of council sizes

# Deduce the councile size using previously introduced conversion to normalizesd rank improvements. 
council_sizes = rdd_data[['gkz_jahr', 'gewinn_norm', 'gewinn', 'listenplatz_norm']].copy()
council_sizes.loc[:,'council_size'] = council_sizes.loc[:,'gewinn'] / council_sizes.loc[:,'gewinn_norm'] *100

# drop rows where gewinn = 0 since we cannot deduct on council sizes in these cases. 
council_sizes.drop(labels=council_sizes.loc[council_sizes.loc[:, 'gewinn']==0,:].index, axis=0, inplace=True)

# Calculate the 95% percentile of council sizes as well as the number of councils that were voted for. 
N_councils = len(council_sizes.drop_duplicates(subset=['gkz_jahr'])['council_size'])
percentile = int(np.percentile(a=council_sizes.drop_duplicates(subset=['gkz_jahr'])['council_size'], q=95, interpolation='lower'))


# plot a histogram for the council sizes
bins = int((council_sizes['council_size']).max() - (council_sizes['council_size']).min())

hist_council_sizes(df=council_sizes, bins=bins)
print('The number of council elections that we have observations for is {}. \nThe 95 % percentile of council sizes is {}.'.format(N_councils, percentile))

Figure 2 displays the rank improvement of a female council candidate and the margin of victory of a female mayor in the preceeding mixed gender mayoral election. Since a lot of observations are stacked on top of each other and to prevent the figure from beeing too cluttered, a random sample of 10 % of all available observations is plotted. Looking at the distributions of observed combinations, a pattern of *horizontally as well as vertically aligned datapoints* is revealed. 

First, since the authors work with a dataset that contains the rank improvements of each female council candidate for a given result for the preceeding mayoral election, discrete changes in the horizontal dimension are to be expected. 

Second, the discrete distributionof values in the vertical dimension (along the ordinate) can be explained when a closer look is taken at the distribution of council sizes: As previously described, rank improvements are normalized using the respective size of the council. While the authors do not provide data on the council sizes, this value can easily be deduced from the given values for absolute and relative gains in list ranks for the candidates. As presented in figure 3, sizes of the 214 observed councils are distributed over relatively few values and are clustered at certain values. In addition, 95 % if all councils have not more than 45 seats. 

Concluding, it seems plausible to observe a discrete distribution of normalized rank improvements, given the discussed distibution of council sizes which will cause the denominator for the value on the ordinate to be the same for a large number of observations.

![Error: Here should be figure 2!](out/figure_2a.png "Figure 2: Sample observations on normalized rank improvements and margin of victory.")

![Error: Here should be figure 3!](out/figure_3a.png "Figure 3: Distribution of council sizes.")



### 2.3. Discontinuity at the Critical Value of the Margin of Victory
Considering figure 2, it is not very obvious if there is a discontinuity around the critical value of margin of victory (i.e. a margin of 0). A jump in values on the ordinate, however, might not be visible in this graphical representation. As discussed previously, observations are not distributed continously but instead show strong discontinuities. This might result in a lot of observations being stacked on top of each other, hence not being represented in above plot. However, this problem should be mitigated by plotting only a small fraction of 10% of all observations.

Summarizing, a discontinuity around the cutoff can still not be ruled out. Hence, in another approach we will have a look at binned data. By plotting data tuples of the mean values of both, the margin of victory as well as the rank improvement, a discontinuity should be revealed (if there is any). This is because stacked observations will still have an impact of equal size on the mean values within each bin.

In the next section, this approach of plotting binned data will be pursued further. This method is a replication of what the authors present in chapter IV.A. of their article (figure 2).

## 3. Replication and Critical Review
### 3.1 Graphical Evidence: RDD Plot
A clear discontinuity around the cutoff value of the margin of victory of 0, however, is hard - if at all - to spot in figure 1 presented above. This might be due to the very noisy representation caused by plotting every single observation. Hence, a more promising approach is presented by the authors in *Figure 2: Rank Improvement of Female Candidates* of the article which is an RDD-plot for binned normalized rank improvements. A replication of this RDD-plot is displayed in figure 3 below.

In this plot, first, the running variable (female mayor candidate margin of victory) is limited o values from -30 to +30 %. This allows to focus on the part of the data which is most relevant for this analysis. Second, the authors decide to bin the observations into clusters of 3 percentage points width of the running variable. Third, local polynomial regression of the underlying observations is added (red line) estimating the relation of both variables. Finally, the pointwise 95 % confidence intervals are illustrated as black lines.



In [ ]:
# Keep only observations that fir above described criteria
rdd_plot_data = rdd_data.loc[(rdd_data.margin_1 >= -30) & (rdd_data.margin_1 <= 30)].copy()

# Create the RDD plot using the function stored in the auxiliary file. 
rslt = rdd_plot(data=rdd_plot_data, x_variable="margin_1", y_variable="gewinn_norm", nbins=20, ylimits=(-6,6), width=20.1, deg=1)

### 3.2 Main Results
#### 3.2.1 Regression Results

Two different approaches are presented: 
(1) relates gender of incumbent mayor to  a measure of performance (endog. variable is normalized rank improvement) of female candidates
(2) relates gender of incumbent mayor to the likelihood that an elected member of the council in the same municipality is female (endog. variabel is a dummy that is 1 for council members that are female).

Both regressions are performed using different bandwidths and functional forms. For this replication, I will focus on the main results which are basis for the discussion in the article.

* a) bandwidths:
    - **optimal CCT bandwidth (20.1)**
    - CCT / 2 (10.05)
    - CCT * 2 (40.2)
    - IK (35.59)
* b) functional form:
    - **linear**
    - quadratic

Only the optimal CCT bandwidth is used for the quadratic specification resulting in five specifications in total. The second approach results in insignificant results while pointestimates seem to be in line with the results of the first approach. Since the subsequent discussion in mainly based on the first specification it will be calculated by default in the following cells. Note that the results for different bandwidths can easily be reproduced by adjusting the *bw* variable in subsequent code cells. 

**Specification 1: Rank Improvements of Female Candidates:**

In this specification normalized rank improvements are endogenous in the margin of victory of a female mayor, a dummy that takes value 1 if a female won the mayor election and an interaction term of both, margin and dummy. The regression model is illustrated in equation (\ref{eq2}) below. In this notation, $i$ denotes one female council candidate. The dummy $D_{f\_mayor, i}$ takes value 1 if the mayor in force at the time of the council election is female. Finally, $MoV$ denotes the *margin of victory* of the female mayor candidate. Note that for values bigger 0 for $MoV$, $D_{f\_mayor, i}$ takes value 1 and vice versa. 

\begin{align}
    normalized\_rank\_improvement_i = \beta_0 + \beta_1 D_{f\_mayor, i} + \beta_2 MoV_i + \beta_3 D_{f\_mayor, i} * MoV_i \label{eq2} \tag{2}
\end{align}

Additionally, observations are weighted using their distance to the cutoff value of zero for the margin of victory. Lower weights are attached to observations farther away from the cutoff value. Thus, observations very close to the cutoff will have a weight close to 1. At the same time, observations where the size of the margin of victory close to the size of the bandwidth will have a weight close to 0. 

In [ ]:
# Set the bandwidth (taken from the paper)
bw = 20.1

# Get regression weights
rdd_data["weight"] = calculate_weights(df=rdd_data, bandwidth=bw)

# Run regression for approach (1): normalized rank improvement of female candidates on gender of mayor
sm.WLS(endog=rdd_data.loc[abs(rdd_data["margin_1"])<bw]["gewinn_norm"], 
       exog=sm.add_constant(rdd_data.loc[abs(rdd_data["margin_1"])<bw][["female_mayor", "margin_1", "inter_1"]]), 
       weights=rdd_data.loc[abs(rdd_data["margin_1"])<bw]["weight"],
      ).fit(cov_type="cluster", 
             cov_kwds={"groups":rdd_data.loc[abs(rdd_data["margin_1"])<bw]["gkz"]}
            ).summary()

Comparing the results presented in the paper (see figure X below) to the regression summary above, it can be seen that the results presented in the article can be replicated. As eluded to earlier, the authors present a number of different specifications for varying bandwidths as well as functional forms. The results of the main model (optimal bandwidth and linear functional form) can be seen in the column denoted with (1) in the figure below. 

The presented results for varying bandwidths can easily be replicated by adjusting the *bw* variable in the code cell above to take the according value in row "Bandwidth size" of figure X. While the resulting estimates as well as the number of observations and other outputs can be replicated for specifications (1) through (4), the quadratic functional form (specification (5)) cannot be estimated due to limitations of the python package used for this estimation. 

The quadratic specification, however, acts merely as another robustness check for the main results. As seen in figure X below, changes in the functional form (i.e. adding the squared version of the third and fourth summand on the right hand side of equation (2)) do not result in substantial changes in the regression outcome. The pointestimate remains at approximatly the same level and still shows to be significant at the 95% level. 

**Figure 4: Main results I: Rank improvement of female candidates as presented by the authors**
![Error: here should be figure 4](img/main_results_1.png)


In [ ]:
# Descriptives of the regression result
N = rdd_data.loc[abs(rdd_data["margin_1"])<bw]["gewinn_norm"].count()
elec = rdd_data.loc[abs(rdd_data["margin_1"])<bw]["gkz_jahr"].nunique()
muni = rdd_data.loc[abs(rdd_data["margin_1"])<bw]["gkz"].nunique()
y_mean = rdd_data.loc[abs(rdd_data["margin_1"])<bw]["gewinn_norm"].mean()
y_sd = rdd_data.loc[abs(rdd_data["margin_1"])<bw]["gewinn_norm"].std()
main_result_descriptive = pd.DataFrame(data=[N, elec, muni, y_mean, y_sd], 
                                       index=["Observations", 
                                              "Elections", 
                                              "Municipalities", 
                                              "Dep. Variable Mean", 
                                              "Dep. Variable SD"
                                             ], 
                                       columns = ["value"],
                                      )
main_result_descriptive

As a check for mistakes in the replication process, the descriptive statistics which the authors provide with the main result (see figure 4 above), are replicated as well. All descriptive values calculated for the data used in the replication process match with their counterparts presented by the authors. Thus, there is no reason to believe that the underlying observations of the regression analysis are different from the ones used by the authors of the article.

The results of this regression analysis indicate that the victory of a female candidate in the mayoral election in a municipality is associated with an improvement of 3.7 normalized ranks. For the median council size of 31 seats this corresponds to approximatly 1.15 ranks.

**Approach 2: Share of Women among Candidates Elected to the Council:**

For the second approach, the gender of an elected council member is used as the endogenous variable while the right side of the regression equation remains the same. Althogh the endogenous variable is a binary variable in this regression equation, the authors apply the same weighted linear regression method which is used in the first approach presented above. Thus, the estimated model can be denoted as follows.

\begin{align}
    D_{gender\_council\_candidate,i} = \beta_0 + \beta_1 D_{f\_mayor, i} + \beta_2 MoV_i + \beta_3 D_{f\_mayor, i} * MoV_i \label{eq3} \tag{3}
\end{align}

Note that the right hand side of equation (3) remains unchanged in comparison to the previously discussed specification presented in equation (2). The binary endogenous variable $D_{gender\_council\_candidate,i}$ takes value 1 if candidate $i$ is female and 0 otherwise. 

At the same time, the data set used for this regression model is slightly different from the one used previously. While it includes data on male as well as female council candidates now, it only contains those of council candidates that end up to be elected into council.

In [ ]:
# Select data on (a) mixed gender mayor elections and (b) council candidates who were elected
candidates_elected = df_main_dataset.loc[(df_main_dataset.rdd_sample == 1) & (df_main_dataset.elected == 1)].copy()

# set same bandwidth as authors do
bw = 23.90

# Add weighing vector as calculated by the authors
candidates_elected["weight"] = calculate_weights(df=candidates_elected, bandwidth=bw)

# Run regression for approach (2): Gender of an elected council candidate
sm.WLS(endog=candidates_elected.loc[abs(candidates_elected["margin_1"])<bw]["female"], 
       exog=sm.add_constant(candidates_elected.loc[abs(candidates_elected["margin_1"])<bw][["female_mayor", "margin_1", "inter_1"]]), 
       weights=candidates_elected.loc[abs(candidates_elected["margin_1"])<bw]["weight"],
      ).fit(cov_type="cluster", 
             cov_kwds={"groups":candidates_elected.loc[abs(candidates_elected["margin_1"])<bw]["gkz"]}
            ).summary()

Again, the results of this regression correspond to the results presented in the article (see figure 5 below). Similarly to the replication of the first approach presented earlier, the regression summary is set to replicate the main results and, thus, uses the optimal bandwidth and a linear functional form as presented in column (1) of figure X below. However, by setting the according value for the *bw* variable, specifications (2) through (4) can be replicated from the code block above effortlessly.

**Figure 5: Main results II: Share of women among candidates elected to the council as presented by the authors**
![Error: here should be figure 5](img/main_results_2.png)

The main result provides a point estimate of 0.041 which indicates that the voctory of a female mayor candidate in a mixed gender election is associated with a 4.1 percentage points increase in the share of female council members.

In [ ]:
# Calculate descriptives
N = candidates_elected.loc[abs(candidates_elected["margin_1"])<bw]["female"].count()
elec = candidates_elected.loc[abs(candidates_elected["margin_1"])<bw]["gkz_jahr"].nunique()
muni = candidates_elected.loc[abs(candidates_elected["margin_1"])<bw]["gkz"].nunique()
y_mean = candidates_elected.loc[abs(candidates_elected["margin_1"])<bw]["female"].mean()
y_sd = candidates_elected.loc[abs(candidates_elected["margin_1"])<bw]["female"].std()
main_result_descriptive = pd.DataFrame(data=[N, elec, muni, y_mean, y_sd], 
                                       index=["Observations", 
                                              "Elections", 
                                              "Municipalities", 
                                              "Dep. Variable Mean", 
                                              "Dep. Variable SD"
                                             ], 
                                       columns = ["value"],
                                      )
main_result_descriptive

#### 3.2.2 Robustness
In order to evaluate the robustness of their baseline results, the authors present two different approaches: First, a placebo test and, second, a test for different scalings of the outcome variable. The authors argue that, overall, the baseline results show to be robust to both of these checks.

##### 3.2.2.1 Placebo Test
The placebo test is described to check whether past values of the outcome variable "can be explained by contemporaneous values of the treatment vaiable". Applied to the analysis presented in the article, this translates to a test for statistically significant correlations between the gender of the mayor in council election period $t$ and the rank improvements of female council candidates in period $t-k$. The authors argue that "if the female mayor dummy has a significant effect on the rank improvements of women in past elections, this would call our baseline estimates into question."

Time lag $k$ is chosen to be five by the authors. This assumption is not discussed in the article, although it is somewhat questionable since the regular period of office of a mayor in Hesse is six years (Hessian Municipal Code, 2015). By setting $k=5$ the authos ensure that a mayor is in office for at most one year before the council election takes place. Thus, mitigating effects on anti-female voters bias arguably are quite low. At the same time, one could argue that the effects of a female mayor on anti-female voters bias might be particularly large right after the mayor's election while media coverage of the mayor might be especially broad.

Indeed, as shown in the next cell, only for 40% of the observations used in the placebo test, the mayor is in charge for less then five years at the moment the council election takes place. In other words, voters have been exposed to the same mayor in the council election in year $t$ that was already in charge during the council election in year $t-5$ in 60% of the observations. Hence, the results of the placebo test as it is presented by the authors might be imprecise.

Next, the placebo test as it is depicted in the article is replicated. Subsequently, a placebo test based on a more strict data selection is carried out. These analyses are conducted using the placebo dataset ("dataset_with_lagged_rank_improvments.dta") which is provided by the authors. This dataset contains information on the mayor's gender and MoV at time $t$ as well as results for the council election in $t-5$.

TODO: Add reference of for mayor: HGO §39 : http://www.lexsoft.de/cgi-bin/lexsoft/justizportal_nrw.cgi?xid=146137,50

TODO: Add reference of for coucil: HGO §36



In [ ]:
# Is the longer office term of mayors a relevant problem?
# If the mayor is in office for 5 or more years at t (time of council election) then
# he/ she was in charge during last council election -> no real placebo test!
# How many mayors where in office for 5 or more years at time t, when t is the year of council election?

# Merge placebo date with data on the sum of years a mayor is in office at year t
df_placebo_2 = pd.merge(left=df_placebo, 
                  right=df_main_dataset[["gkz_jahr", "sum_years_as_mayor"]].drop_duplicates(), 
                  on=["gkz_jahr"], 
                  how="left")

# calculate share of observations with a mayor that is in office for less
# then 5 years in all observations used in placebo test
temp = np.round(len(df_placebo_2.loc[df_placebo_2["sum_years_as_mayor"]<5,:].index) / len(df_placebo_2.index),3)
print("The mayor in office is in charge for less than five years for {}%".format(temp)+
      " of the observations on the placebo dataset.")

In [ ]:
# Replicate the placebo test as it is performed and presented in the article

# Set the bandwidth (taken from the paper)
bw = 15.71 # CCT
#bw = 15.71/2
#bw = 15.71*2
#bw = 17.22 # KI

# Get regression weights
df_placebo["weight"] = calculate_weights(df=df_placebo, bandwidth=bw)

# Run regression of rank improvements in t-5 on mayor election result in t
sm.WLS(endog=df_placebo.loc[abs(df_placebo["margin_1"])<bw]["gewinn_norm"], 
       exog=sm.add_constant(df_placebo.loc[abs(df_placebo["margin_1"])<bw][["female_mayor", "margin_1", "inter_1"]]), 
       weights=df_placebo.loc[abs(df_placebo["margin_1"])<bw]["weight"],
      ).fit(cov_type="cluster", 
             cov_kwds={"groups":df_placebo.loc[abs(df_placebo["margin_1"])<bw]["gkz"]}
            ).summary()

In [ ]:
# placebo test with only those observations from placebo dataframe in which the mayor in in office
# for less than 5 years
data = df_placebo_2.loc[df_placebo_2["sum_years_as_mayor"]<5]

# Set the bandwidth (chosen to be the same as in the authors approach)
#bw = 15.71 # CCT
#bw = 15.71/2 
#bw = 15.71*2 
#bw = 17.22 # IK

# Get regression weights
data["weight"] = calculate_weights(df=df_placebo_2, bandwidth=bw)

# Replicate the placebo test using only these observations
sm.WLS(endog=data.loc[abs(data["margin_1"])<bw]["gewinn_norm"], 
       exog=sm.add_constant(data.loc[abs(data["margin_1"])<bw][["female_mayor", "margin_1", "inter_1"]]), 
       weights=data.loc[abs(data["margin_1"])<bw]["weight"],
      ).fit(cov_type="cluster", 
             cov_kwds={"groups":data.loc[abs(data["margin_1"])<bw]["gkz"]}
            ).summary()

Summarizing the results of the placebo test, the following can be concluded:
First, the estimation restults of a placebo test as it is presented int he article, particularly in table A.8 of the online appendix (see figure 6 below), can be replicated. In accordance to the argument in the article, the point estimate of this estimation shows to be relatively small in comparison to the main results. But more importantly, they do not show statistical significance and even reveal changes in signs for different bandwidths. This is seen as supporting evidence for robustness of the main results. 


**Figure 6: Robustness Test I: Rank improvement of female candidates in previous council election**
![Error: here should be figure 6](img/robustness_1.png)

Second, following above discussion of the term duration of mayors in Hesse and, consequentially, excluding all observations for which the mayor at time $t$ was in office for five or more years, does not lead to substantial differences in results. While in this specification there is no change in signs of the point estimate of the female mayor dummy, estimates still do not show statistical significance. 

##### 3.2.2.2 Scaling of the Outcome Variable

In the second robustness test, the authors test whether the results of their analysis are sensitive regarding to changes in the scaling of the outcome variable, i.e. the rank improvements of council candidates. As argued in section XXX, the authors use normalized rank improvements in order to facilitate interpretability of the results as well comparability of changes in rank for different council sizes. Assessing robustness with regard to this scaling, alternative specifications of the regression model are presented using (a) raw rank improvements and (b) a dummy variable that takes value 1 if a candidate experienced a strictly positive rank improvement. Estimations for both specifications will be replicated in subceeding cells.

In [ ]:
# (a) raw rank improvements instead of normalized rank improvements as endogenous variable
# Set the bandwidth (taken from the paper)
bw = 19.31

# Get regression weights
rdd_data["weight"] = calculate_weights(df=rdd_data, bandwidth=bw)

# Run regression for approach (1): normalized rank improvement of female candidates on gender of mayor
sm.WLS(endog=rdd_data.loc[abs(rdd_data["margin_1"])<bw]["gewinn"], 
       exog=sm.add_constant(rdd_data.loc[abs(rdd_data["margin_1"])<bw][["female_mayor", "margin_1", "inter_1"]]), 
       weights=rdd_data.loc[abs(rdd_data["margin_1"])<bw]["weight"],
      ).fit(cov_type="cluster", 
             cov_kwds={"groups":rdd_data.loc[abs(rdd_data["margin_1"])<bw]["gkz"]}
            ).summary()

In [ ]:
# (b) rank improvement dummy instead of normalized rank improvement as endogenous variable
# Set the bandwidth (taken from the paper)
bw = 18.11

# Get regression weights
rdd_data["weight"] = calculate_weights(df=rdd_data, bandwidth=bw)

# Run regression for approach (1): normalized rank improvement of female candidates on gender of mayor
sm.WLS(endog=rdd_data.loc[abs(rdd_data["margin_1"])<bw]["gewinn_dummy"], 
       exog=sm.add_constant(rdd_data.loc[abs(rdd_data["margin_1"])<bw][["female_mayor", "margin_1", "inter_1"]]), 
       weights=rdd_data.loc[abs(rdd_data["margin_1"])<bw]["weight"],
      ).fit(cov_type="cluster", 
             cov_kwds={"groups":rdd_data.loc[abs(rdd_data["margin_1"])<bw]["gkz"]}
            ).summary()

Comparing the results of above analysis to the results presented in the article (figure A.10 of the online appendix, shown in figure 7 below), no differences can be noticed. The replication of this test for rebustness of the main results with regard to scaling of the outcome variable, thus, can be replicated.

**Figure 7: Robustness Test II: Alternative scalings of rank improvement**
![Error: here should be figure 7](img/robustness_2.png)

### 3.3 Internal Validity of the RD Design
Establishing internal validity is a prerequisite of the regression discontinuity design to be able to identify treatment effects in the same way a randomized trial does (Jacob & Zai 2012). Therefore, it is crucial to exclude a non-random distribution of observations around the cutoff value of the treatment variable. In this scenario this means that municipalities around the cutoff value in the margin of victory of the mayor must not systematically differ in their characteristics. 

In other words, we need to rule out other drivers and channels through which a correlation between the rating variable (i.e. the margin of victory) and the outcome (rank improvements) might arise.

#### 3.3.1 Pretreatment Municipality Characteristics
The first test of validity the authors present adresses whether the RD design truly achieves a local randomization around the cutoff. If this is the case, then both, municipalities that are just below the cutoff of MoV = 0 and those just above this threshold should show the same distribution of characteristics. This is investigated using a t-test for differences in means of these chracteristics in the year before the mayor election. Subsequently, a regression analysis is used to test for a correlation of the municipalities characteristics prior the mayor election (hence the name "pretreatment") and the elected mayors' genders. If observations are truly randomized around the cutoff value we should not see any correlation between these characteristics and the mayors' genders.

In a first step, we will have a look at the municipalities' characteristics one year before a mayor election and perform a t-test to explore whether there are differences between municipalities in which a male candidate won the mixed-gender election and municipalities in which a female won the mixed-gender mayor election. This comparison will be performed for all mixed-gender elections as well as for close those mixed-gender elections only (i.e., with a margin of victory below 10%).


In [ ]:
# t-test for equality of means of muni's characteristics

# Merge data on mayor elections with municipalities' characteristics in the year prior the mayor election
# only keep observations on mixed gender elections
df_pretreatment_characteristics = df_mayor_election_data.loc[df_mayor_election_data.rdd_sample == 1].copy()

# merge with muni's characteristics in previous year
# comment out -1 in following line to get the same results the authors report.
df_pretreatment_characteristics["char_year"] = df_pretreatment_characteristics.jahr -1
df_pretreatment_characteristics = df_pretreatment_characteristics.merge(right=df_municipality_characteristics, 
                                      how="left", 
                                      left_on=["gkz", "char_year"], 
                                      right_on=["gkz", "jahr"], 

)
# We want to perform t-tests twice: for all mixed-gender elections and only "close" ones with MoV < 10 %
bandwidths = [100, 10]

# get results from the function in the auxiliary file
rslt_dict = ttest_mean_characteristics(df=df_pretreatment_characteristics, bw=bandwidths)

for k in rslt_dict.keys():
    print("For a margin of victory of " + str(k) + " or less :")
    display(rslt_dict[k])

In comparison to the results of this analysis as they are presented by the authors in table A4 (see figure 8 below), small differnces can be seen. It is possible to replicate the results as presented by the authors if, instead of considering the municipalities characteristics of the *year preceeding* the mayor election, one considers the characteristics of the *same year* as the mayor election. The results shown in this notebook, however, are calculated in accordance to the intuition discussed earlier: We want to find out, whether the RD design truly results in a local randomization, hence the municipalities closely around the cutoff must not significantly differ in characteristics that might have an impact on the outcome of a mixed gender mayor election. 

Considering municipalities' characteristics of the year preceeding the election will result in slightly different results than the ones presented by the authors. The authors argue, that when considering 
>all mixed-gender races, the share of women in local public employment \[row log_female_sh_gem_besch\] is 8 percent higher, while in the manufacturing sector \[row log_female_share_prod\] it is 8 percent lower in municipalities with female mayors. For close mixed-gender elections, we find only one significant difference: municipalities with female mayors have larger populations \[row log_bevoelkerung\] than municipalities with male mayors. Yet, the effect is only barely significant and may be explained by the highly skewed distribution of city sizes.

This replication is able to confirm these results with regard to the close mixed-gender elections. For all mixed-gender elections, however, no significant differences can be found comparing characteristics for one year before the election. Results of this test for differences in mean shows to be quite sensitive with respect to small changes in the year for which characteristics are observed. Particularly, estimations including all mixed-gender mayor elections change drastically, showing significance for different characteristics for changes in the year for which these charactersitics are observed. This limits robustness of these results, especially since one should not expect a characteristic like the share of women in the manufacturing sector to change very rapidly from one year to the next.

However, when assessing whether the local randomization around the cutoff truly is achieved, differences in characteristics of close mixed-gender elections should be investigated in greater detail. Thus, the significant difference in population sizes of municipalities closely on either side of the cutoff really is what should be concerning when evaluating the internal validity of the RD design. Additionally, this difference shows to be consistent for changes in the year that is observed. Nevertheless, the authors argue that: 
>"Yet, the effect is only barely significant and may be explained by the highly skewed distribution of city sizes."

Overall, mixed results can be drawn from this analysis. While the authors argue that differences are rather small and might arise from a skewed distribution, it cannot be precluded that differences in characteristics (i.e. population sizes) have an impact on the distribution around the cutoff. This, particularly, holds for municipalities closely around the cutoff value. In case population size truly impacts the rating of municipalities around the threshold, the local randomization cannot be held on to. In consequence the RD design is likely to yield biased estimates for the causal impact of the treatment (Lee 2008). For this reason, I present further investigations on this aspect of local randomization in section 4 of this notebook, succeeding the replication of the article.

**Figure 8: Validity test I: Differences in municipalitiy characteristics with female and male mayors**
![Error: here should be figure 8](img/validity_test_1.png)

Subsequently, the authors analyze whether  differences in municipalities are able to explain a discontinuity in rank improvements of female council candidates. This is done in two consecutive steps:
1. Municipalities' characteristics are used to predict normalized rank improvements of council candidates.
&rarr; See whether municipalities' characteristics have explanatory power on rank improvements.
2. These prediced rank improvements are used as an endogenous variable in a RDD regression.
&rarr; Can municipalities' characteristics explain a discontinuity in rank improvements?

Both these steps are replicated in the following cells.

First, municipalities' characteristics are used to predict normalized rank improvement. The calculations in the article are resembled exactly by including all observations (*bw = 100*). However, by adjusting the *bw* variable, this can easily be changed. In fact, it might by more relevant to check whether rank improvements can be predicted for municipalities in which the margin of victory is close to the cutoff value for two resons:

1. We want to preclude that differences in charactersitics of municipalities lead to a non-random local distribution of municipalities around the cutoff.
  
2. We say earlier, that differences in municipalities' characteristics show more significance for close mixed-gender elections (i.e., those that end up closely to the cutoff) than for all mixed-gender elections.

In contrast to the analysis presented in the article, in particular table A5 of the online appendix, the following estimation, thus, does not include all mixed-gender elections but focusses of those with an absolute margin of victory (MoV) smaller then 10. 

In [ ]:
# Define bandwidth of data around the cutoff value (measured in absolute values)
bw=10

# define working df that does not contain any missing values and only observations within given bw
df_predicted_rank_improvements = rdd_data.loc[abs(rdd_data["margin_1"])<bw].dropna(subset=["margin_1", 
                                                          "gewinn_norm", 
                                                          "log_bevoelkerung", 
                                                          "log_flaeche", 
                                                          "log_debt_pc", 
                                                          "log_tottaxrev_pc", 
                                                          "log_gemeinde_beschaef_pc", 
                                                          "log_female_sh_gem_besch", 
                                                          "log_tot_beschaeft_pc", 
                                                          "log_female_share_totbesch", 
                                                          "log_prod_share_tot", 
                                                          "log_female_share_prod"
                                                         ]
                                                )

# Run regression of rank improvements on municipalities' characteristics
muni_characteristics = sm.OLS(endog=df_predicted_rank_improvements["gewinn_norm"], 
       exog=df_predicted_rank_improvements[["log_bevoelkerung", 
                      "log_flaeche", 
                      "log_debt_pc", 
                      "log_tottaxrev_pc", 
                      "log_gemeinde_beschaef_pc", 
                      "log_female_sh_gem_besch", 
                      "log_tot_beschaeft_pc", 
                      "log_female_share_totbesch", 
                      "log_prod_share_tot", 
                      "log_female_share_prod"
                     ]]       
      ).fit()

# Add the predicted rank improvement to the dataframe as it is needed in subceeding steps
df_predicted_rank_improvements["pred_rank_improvement"] = muni_characteristics.predict()

# Show regression summary
muni_characteristics.summary()


### TODO: Here might be room for improvement: Use a lasso regression to get rid of the noise introduced by adding a llot of regressors
### which do not seem to be relevant. Maybe this will lead to a better fit. 

Two characteristics of the municipalities reveal to be significantly correlated with the rank improvement of a female candidate: Population size of the municipality (10% level) and per capita government employees (1% level). In addition, the regressor tax revenues per capita just barely misses to 10% significance threshold.  

While the population size appears to be significantly positive correlated with rank improvement, the effect is quite small and does not seem to be of high economic significance: An increase of 1 % in population is associated with an increase in rank improvement of a female council candidate of approximatly 0.62/100 normalized ranks. For the median size of a council (31 seats) this corresponds to 0.192 ranks.
Similarly, an increase in government employees is associated with an positive rank improvement of a female council candidate of approximalty 0.490 ranks for the median council size.

In the second step, the authors use the prediction for rank improvements from above OLS model and test whether a discontinuity around the cutoff value of a margin of victory for a female mayor can be determined.

In [ ]:
# Use predicted values from above regression as endogenous variable in a regresion of female mayor on 
# rank improvements (main model).

# Keep only 1 observation for each council election, since muni's characteristics dont vary within that
df_predicted_rank_improvements.drop_duplicates(subset=["gkz", "pred_rank_improvement"], inplace=True)

# Set the bandwidth (taken from the paper)
bw = 18.81


# Get regression weights
df_predicted_rank_improvements["weight"] = calculate_weights(df=df_predicted_rank_improvements, bandwidth=bw)

# Run regression for predicted normalized rank improvement of female candidates on gender of mayor
sm.WLS(endog=df_predicted_rank_improvements.loc[
    abs(df_predicted_rank_improvements["margin_1"])<bw]["pred_rank_improvement"
        ], 
       exog=sm.add_constant(df_predicted_rank_improvements.loc[abs(df_predicted_rank_improvements["margin_1"])<bw][[
           "female_mayor", "margin_1", "inter_1"
       ]]), 
       weights=df_predicted_rank_improvements.loc[abs(df_predicted_rank_improvements["margin_1"])<bw]["weight"],
      ).fit().summary()

At this point, noticable differences in results arise between the article and this replication. As seen in figure X below, the authors do not find a statistically significant estimate for the *female_mayor* dummy. However, the speification presented above does indicate a significant discontinuity in predicted normalized rank improvements around the cutoff at the 1% significance level. Indeed, the point estimate for the female_mayor dummy results to be approximatly 2.63, which translates to circa 0.82 ranks for the median council size. To illustrate the scale of this effect: This amounts to approximatly 71% of the size of the estimated effect for the main model (see section 2.2.1: Regression Results).

Summarizing, the presented results differ from those presented in the article by changing the bandwidth of data included. While the authors find no significant discontinuity and, thus, do not reject a successful local randomization by the RD design, the results presented in this notebook cannot support this conculsion. In contrast to the conclusions drawn in the article, this analysis does not preclude a non-random distribution of the rating variable around the cutoff value. Indeed, these results indicate that municipalities' charatersitics for the *year before* the close mixed-gender election happened (i.e. *pretreatment*) are associated with the rank improvements of a female council candidate.

#### 3.3.2 Discontinuity in Density

The next test with regard to internal validity that is presented by the authors is a test for a "discotinuity in density of the running variable at the threshold". That is, if there is a discontinuity in margins of victory that are closely around the cutoff value of 0. The result of this test can be seen in figure 9 below (figure A.2 in the article's online appendex). It is little surprising that a discontinuity cannot be found. Indeed, it is highly implausible that a council candidate can choose into treatment by raising the MoV of a mayor candidate to be just above the threshold of 0.

**Figure 9: Validity test II: McCrary density plot**
![Error: here should be figure 9](img/validity_test_2.png)

In a related validity check, the authors test whether men and women are equally likely to win a mixed-gender mayor election. Particulary, if the likelihood of a victory is equal for close mixed-gender elections. The results as they are presented by the authors in table A.6 of the online appendix can be seen in figure 10 below.

**Figure 10: Validity test II: McCrary density plot**
![Error: here should be figure 10](img/validity_test_3.png)

In [ ]:
# Calculate the a priori probabilities of winning a mixed-gender election 
# as the share of elections they where actually won by each gender.
rslt_dict = {}
bandwidths = [100, 25, 10]

# Loop over different bandwiths of data to consider
for bw in bandwidths:
    # Select data
    data = df_mayor_election_data.loc[
        (df_mayor_election_data["rdd_sample"]==1) & (abs(df_mayor_election_data["margin_1"])<=bw)
    ]
    
    # Write mean, difference in means, N and p-value of the t-test into a frame
    container = pd.DataFrame(index=[1])
    container["Mean female win"] = (data["female_mayor"].sum() / data.female_mayor.count()).round(3)
    container["Diff."] = (2 * container["Mean female win"] - 1).round(3)
    container["Obs."] = data["female_mayor"].count()
    satistic, p_value = ttest_ind(data["female_mayor"], data["male_mayor"])
    container["t_test_p"] = p_value.round(3)
    
    # return results in a dictionary
    rslt_dict[bw] = container

# Print resulting dict in an understanable way
for k in rslt_dict.keys():
    print("t-Test for differences in means for a bandwidth of {}".format(k))
    display(rslt_dict[k])

The results of this validity test can be replicated. Just like presented by the authors, there is significant differences in the probability of a woman to win the mayoral election for large bandwidths, i.e. when not only close mixed-gender elections are considered. The significance in this difference, however, can not be seen for mixed-gender elctions with a MoV of 10% or lower. Thus, there is no reason to believe that either gender has a higher chance of winning close elections. This supports the hyotheses of a local randomization around the cutoff.

#### 3.3.3. Confoundness with Mayor Ideology
The final concern the authors adress in the regard of internal validity are confounding effects due the political identity of mayors and council candidates. This might arise if males and females differ in their party affilition. For example, females politicians might be more likely to be associated with left-wing parties. In this case, the success of a female council candidate in a municipality with a female mayor might be driven by their similar political affiliation  and trends in general popularity of political ideologies, rather than a decrease in anti-female voters bias.

In the article, this is approached by testing for differences in ideology of female and male mayors. Results of this test are provided in table A.7 of the online appendix and can be found in figure 11 below. Again, the authors investigate both, the whole sample of mixed-gender mayor elections and a subsample of those with a MoV smaller 10%. Overall, differences in the ideology of mayors in mixed-gender elections do not show statistical significance. Hence, it is implausible to argue that election outcomes of council candidates are related to these differences.

**Figure 11: Validity test III: Differences in ideology of female and male mayors**
![Error: here should be figure 11](img/validity_test_4.png)

In [ ]:
# Run a t-test for differences in the shares of party affiliation for male and female 
# mayors that won a mixed-gender election

# Set the list of bandwiths for which these differences should be calculated
bandwidths = [100, 25, 10]

# get results from the function in the auxiliary file
rslt_dict = party_affiliation(df=df_mayor_election_data, bw=bandwidths)
    
# Print resulting dict in an understanable way
for k in rslt_dict.keys():
    print("t-Test for differences in shares of mayors that are affilited with each party for a bandwidth of {}".format(k))
    display(rslt_dict[k])

First of all, the results regarding differences in party affiliation of mayors of different gender as presented in the article can be replicated. From these results it can be seen that independent of the bandwidth selection around the cutoff value, mayors reveal very little differences in their party affiliation. In addidtion, none of these differences prove to be statistically significant. 

Building on this result, one can argue that differences in political ideology between male and female politicians are not driving the observed association of mayors that won a close mixed-gender election with rank improvements of council candidates of the same sex. Thus, a confoundedness of mayors' and council candidates' ideologies can be ruled out as an explanation of the main results presented in section 2.2.1.

## 4. Independent Contribution: On Internal Validity of the RD Design
Validity of the RD design as it is presented in the replicated article is discussed in section 3 of this notebook. In this context, I argue that significant differences in population sizes of municipalities closely below and closely above the threshold value of MoV = 0% might be a reason to question whether the RD design truly achieves a local randomization. In this context it is shown that means of population sizes significantly differ between both sides of the threshold.

Furthermore, using municipalities's characteristics to predict rank improvements of council candidates (section 3.3.1) revealed a significant positive correlation between population size and rank improvements. This resulted from an estimation that only included observations that were within a 10 percentage point range around the cutoff value of MoV.

A local randomization with regard to population sizes does not seem very convincing. Therefore, an alternative estimation model is presented in the following which includes population size as a predictor. The model follows equation (4) which is equivalent to equation (2) appended by the population of the municipality in logs as well as an interaction term between the female mayor dummy and population in logs. 

\begin{align}
    normalized\_rank\_improvement_i = \beta_0 + \beta_1 D_{f\_mayor, i} + \beta_2 MoV_i + \beta_3 D_{f\_mayor, i} * MoV_i  + \\ \beta_4 log(population_i) + \beta_5 D_{f\_mayor, i} * log(population_i) \tag{4}
\end{align}

The estimation of this model follows the same WLS method as the estimation of the baseline results presented in section 3.2. A summary of the results of this regression can be seen below.

In [ ]:
# Add intercept of population size and MoV to regression data
df_population_regression = rdd_data.copy()
df_population_regression["inter_margin_pop"] = df_population_regression["female_mayor"] * df_population_regression["log_bevoelkerung"]

# Set the bandwidth 
#bw = 20
#bw = 15
#bw = 10

# Get regression weights
df_population_regression["weight"] = calculate_weights(df=df_population_regression, bandwidth=bw)

# Run regression for approach (1): normalized rank improvement of female candidates on gender of mayor
sm.WLS(endog=df_population_regression.loc[abs(df_population_regression["margin_1"])<bw]["gewinn_norm"], 
       exog=sm.add_constant(df_population_regression.loc[abs(df_population_regression["margin_1"])<bw][
           ["female_mayor", "margin_1", "inter_1", "log_bevoelkerung", "inter_margin_pop"]
       ]), 
       weights=df_population_regression.loc[abs(df_population_regression["margin_1"])<bw]["weight"],
      ).fit(cov_type="cluster", 
             cov_kwds={"groups":df_population_regression.loc[abs(df_population_regression["margin_1"])<bw]["gkz"]}
            ).summary()

The estimation results of this model reveal noticable differences to the results presented for the baseline model. First, the estimation for the female mayor dummy is not significant in this estimation. Hence, the discontinuity in rank improvements at the cutoff is not significant for this estimation. Second, the only predictor that is significantly correlated with the outcome is the population size of the muicipality. Third, the adjusted R-Squared is increased by approximatly 50% in comparison to the baseline model. Changes in the bandwidth result in changes in the adjusted R-squared. However, this 50% increase appears to mark a lower bound.

Since a python package for optimal bandwidth selection is not available, estimation results were calculated using an array of different bandwidths. This can be adjusted by modifying above cell. The currently chosen bandwidth of 15 is used since it fits in the range of different bandwidths that are presented for the authors for different estimations, which generally was from 15 to 20. These results are very little affected by changes in the bandwidth.

To summarize, a comparison of characteristics of municipalities that lie respectively just below or above the cutoff value indicates that a local randomization might not have been achieved. In particular, population size appears to have explanatory power on the election success of female council candidates. Including population size as a covariate in the estimation model results in a non-significant discontinuity while increasing the adjusted R-Squared. Finally, without a random assignment into treated and untreated groups around the cutoff, the internal validity of an RD design must be taken into question, hence causal inference looses credibility in the RD design (see Lee, 2008 for a discussion).